# Import Libs

In [76]:
import re
import os

# Extraindo Personagens

## Descobrindo nomes arquivos de entrada

In [77]:
def list_path_temporadas() -> list:
    dir_dados = 'data'
    diretorios = os.listdir(dir_dados)
    diretorios_temporadas = [f'{dir_dados}/{f}' for f in diretorios if os.path.isdir(f'{dir_dados}/{f}')]
    return diretorios_temporadas

In [78]:
def list_path_epi(path_temporada) -> list:
    diretorios = os.listdir(path_temporada)
    path_episodios = [f'{path_temporada}/{f}' for f in diretorios if os.path.isfile(f'{path_temporada}/{f}')]
    return path_episodios

# Extrair Personagens

In [79]:
def extrair_personagens_episodio(path_arquivo, personagens, personagens_figurantes):
    with open(path_arquivo, 'r') as file:
        for line in file:
            personagem = re.search('^[^"\[]*?:', line.upper())

            if personagem:
                personagem_str = personagem.group(0)
                p = re.compile('((\(.*\))*)(\s?):')
                personagem_str = p.sub('', personagem_str) 
                p = re.compile('\s$')
                personagem_str = p.sub('', personagem_str)
                
                contains_digit = any(char.isdigit() for char in personagem_str)
                
                if contains_digit:
                    p = re.compile('(\s?).(\d)')
                    personagem_figurante = p.sub('', personagem_str)
                    personagens_figurantes.add(personagem_figurante)
                else:
                    x = re.search('((^ALL\s)|(^ALL$)|(^AN?\s))', personagem_str)
                    if not x:
                        personagens.add(personagem_str)

In [80]:
def extrair_personagens():
    personagens = set()
    
    personagens_figurantes = set()
    
    for path_temporada in list_path_temporadas():
        for path_episodio in list_path_epi(path_temporada):
            extrair_personagens_episodio(path_episodio, personagens, personagens_figurantes)
            
    personagens_principais = set()
    for personagem in personagens:
        if 'YOUNG ' in personagem:
            personagens_principais.add(personagem.replace('YOUNG ', ''))
            continue
            
        if personagem not in personagens_figurantes:
            personagens_principais.add(personagem)    
             
    return personagens_principais

In [81]:
personagens = extrair_personagens()

In [82]:
personagens

{'ADDAM MARBRAND',
 'AEMON',
 'AERON',
 'AERSON',
 'AHSA',
 'ALLISER',
 'ALLISER THORN',
 'ALLISER THORNE',
 'ALLISER THRONE',
 'ALLISTER',
 'ALTON',
 'AMORY',
 'ANGUY',
 'ANNOUNCER',
 'ARAY',
 'ARCHERS',
 'AREO',
 'ARMORY',
 'ARTHUR',
 'ARYA',
 'ARYA STARK',
 'ASSASSIN',
 'ATTENDANT',
 'AXELL FLORENT',
 'AYRA',
 'BABY',
 'BAELISH',
 'BALON',
 'BALON DWARF',
 'BANKER',
 'BARRISTAN',
 'BARRISTAN SELMY',
 'BARRISTON',
 'BELICHO',
 'BENJEN',
 'BENJEN STARK',
 'BERIC',
 'BERIC DONDARRION',
 'BIANCA',
 'BITER',
 'BLACK HAIRED PROSTITUTE',
 'BLACK LORREN',
 'BLACK WALDER',
 'BLACKFISH',
 'BLACKSMITH',
 'BLONDE PROSTITUTE',
 'BOBONO',
 'BOLTON BANNERMAN',
 'BOLTON GUARD',
 'BOLTON OFFICER',
 'BOTH',
 'BOTH JON AND SAM',
 'BOWEN MARSH',
 'BOY',
 'BRAAVOSI MAN',
 'BRAN',
 'BRAN STARK',
 "BRAN'S VOICE",
 'BRAND',
 'BRANN STARK',
 'BRIENNE',
 'BRINENNE',
 'BRON',
 'BRONN',
 'BROTHEL KEEPER',
 'BROTHER',
 'BROTHERS',
 'BRYNDEN',
 'BUER',
 'BUYER',
 'BYSTANDERS',
 'CAMELLO',
 'CAPTAIN',
 'CASSEL',
